In [2]:
%pip install firecracker

import platform

if platform.system() != "Linux":
	print("Firecracker requires Linux (termios is not available on Windows). Use a Linux/WSL kernel to run this cell.")
else:
	from firecracker import MicroVM

	# Create a new microVM instance
	microvm = MicroVM(name="agent-framework-vm", vcpu=2, memory_size=512)

	microvm.status()

Note: you may need to restart the kernel to use updated packages.
Firecracker requires Linux (termios is not available on Windows). Use a Linux/WSL kernel to run this cell.


c:\Users\chavv\github\agent-framework\.venv\Scripts\python.exe: No module named pip
